In [1]:
# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature


import os
import sys
import json
from zipfile import ZipFile
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install -U hda

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import tqdm
from hda import Client, Configuration

In [4]:
dataset_id = "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS_MONTHLY_MEANS"

In [5]:
{
  "dataset_id": "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS_MONTHLY_MEANS",
  "itemsPerPage": 200,
  "startIndex": 0
}

{'dataset_id': 'EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS_MONTHLY_MEANS',
 'itemsPerPage': 200,
 'startIndex': 0}

# Data input 1 (Total Precipitation and Snowfall)

In [6]:
data = {
  "dataset_id": "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS_MONTHLY_MEANS",
  "product_type": [
    "monthly_averaged_reanalysis"
  ],
  "variable": [
    "Total Precipitation" , "Snowfall"
  ],
  "year": [
    "1979",
    "1980",
    "1981",
    "1982",
    "1983",
    "1984",
    "1985",
    "1986",
    "1987",
    "1988",
    "1989",
    "1990",
    "1991",
    "1992",
    "1993",
    "1994",
    "1995",
    "1996",
    "1997",
    "1998",
    "1999",
    "2000",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023"
      ],
  
  "month": [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10", 
    "11",
    "12"],
  "time": [
    "00:00",
    "06:00",
    "12:00",
    "18:00"
  ],
  "bbox": [
    -180,
    90,
    180,
    60
  ],
  "format": "netcdf",
  "itemsPerPage": 200,
  "startIndex": 0
}

In [ ]:
#def load_data():
conf = Configuration(user = "zcasty", password = "green1234Zaria")
    #hda_client = Client(config = conf)
    #c = Client(debug=True)
c = Client(config=conf, max_workers=12)
matches = c.search(data)
print(matches)
matches.download()

In [ ]:
# get filename of latest .nc downloaded file 
files = [file for file in os.listdir(".") if (file.lower().endswith('.nc'))]
list_nc_file = []

for file in sorted(files,key=os.path.getmtime, reverse=True):
    list_nc_file.append(file)
    
print(f'Selected netCDF file: {list_nc_file[0]}')  

In [ ]:
# Define the path to your NetCDF file
file_path = "reanalysis-era5-single-levels-monthly-means-d891cf072f02d7845c0ec8350ca6478a.nc"

# Load NetCDF file
ds = xr.open_dataset(file_path)

# Print dataset to understand its structure
print(ds)

# Extract the Total Precipitation and Snowfall variables
tp = ds['tp']  # total precipitation variable
sf = ds['sf']  # snowfall variable

In [ ]:
tp

In [ ]:
sf

# Data input 2 (Large Scale and Convective)

In [ ]:
data = {
  "dataset_id": "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS_MONTHLY_MEANS",
  "product_type": [
    "monthly_averaged_reanalysis"
  ],
  "variable": [
    "large_scale_precipitation", "convective_precipitation"
  ],
  "year": [
    "1979",
    "1980",
    "1981",
    "1982",
    "1983",
    "1984",
    "1985",
    "1986",
    "1987",
    "1988",
    "1989",
    "1990",
    "1991",
    "1992",
    "1993",
    "1994",
    "1995",
    "1996",
    "1997",
    "1998",
    "1999",
    "2000",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023"
      ],
  
  "month": [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10", 
    "11",
    "12"],
  "time": [
    "00:00",
    "06:00",
    "12:00",
    "18:00"
  ],
  "bbox": [
    -180,
    90,
    180,
    60
  ],
  "format": "netcdf",
  "itemsPerPage": 200,
  "startIndex": 0
}

In [ ]:
# Calculate liquid precipitation
liquid_precip = tp - sf

# Print the result to verify
print(liquid_precip)

In [ ]:
# North Polar Orthographic projection with contour lines
def plot_map(ax, data, title, vmin=0, vmax=2000, cmap='PuBu', contour_levels=10):
    ax.set_global()
    ax.coastlines()
    
    # Circular boundary for the polar projection
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    # Set map extent
    ax.set_extent([-180, 180, 66.5, 90], crs=ccrs.PlateCarree())

    # Multiply the data by 1000 to convert from meters to millimeters
    data_mm = data * 1000 * 1000

    # Plot the data
    pp = ax.pcolormesh(data_mm.longitude, data_mm.latitude, data_mm.mean(dim='date'),
                       cmap=cmap, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)

    # Add contour lines on top of the plot
    contours = ax.contour(data_mm.longitude, data_mm.latitude, data_mm.mean(dim='date'), 
                          levels=contour_levels, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())

    # Add colorbar
    cbar = plt.colorbar(pp, ax=ax, orientation='horizontal', pad=0.09)
    cbar.set_label('mm of water equivalent')

    # Add title
    ax.set_title(title)

# Create figure and subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={'projection': ccrs.Orthographic(central_latitude=90)})

# Plot Total Precipitation with contours
plot_map(axes[0], tp, 'Total Precipitation (1979-2023)', vmin=0, vmax=2000, contour_levels=np.linspace(0, 2000, 10))

# Plot Snowfall with contours
plot_map(axes[1], sf, 'Snowfall (1979-2023)', vmin=0, vmax=2000, cmap='cool', contour_levels=np.linspace(0, 2000, 10))

# Plot Calculated Liquid Precipitation (TP - SF) with contours
plot_map(axes[2], liquid_precip, 'Liquid Precipitation (TP - SF, 1979-2023)', vmin=0, vmax=2000, cmap='Purples', contour_levels=np.linspace(0, 2000, 10))

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()
plt.savefig('MeanArcticTP_S_LP.png', dpi=300, bbox_inches='tight')  # Save the figure
